In [1]:
import math

In [1]:
class BPVal:
    def __init__(self, value):
        self.value = float(value)
        self._previous = []
        self.grad = 0.0
        self._backpropagation = lambda: None

    def __repr__(self):
        return f'BPVal({self.value})'

    def __add__(self, other):
        other = other if isinstance(other, BPVal) else BPVal(other)
        newValue = BPVal(self.value + other.value)
        newValue._previous = [self, other]

        def backprop():
            self.grad += newValue.grad
            other.grad += newValue.grad
        newValue._backpropagation = backprop
        
        return newValue

    def __radd__(self, other):
        return self + other
        
    def __mul__(self, other):
        other = other if isinstance(other, BPVal) else BPVal(other)
        newValue = BPVal(self.value * other.value)
        newValue._previous = [self, other]

        def backprop():
            self.grad += other.value * newValue.grad
            other.grad += self.value * newValue.grad
        newValue._backpropagation = backprop
        
        return newValue

    def __rmul__(self, other):
        return self * other

    def __truediv__(self, other):
        return self * (other **(-1))

    def __rtruediv__(self, other):
        return (self**(-1)) * BPVal(other)

   # def __eq__(self, other):
    #    return self.value == other.value    #breaks hashing so you cant use set without implementig hash?!

    #def __lt__(self, other):
     #   return self.value < other.value

    def __sub__(self, other):
        return self + -other

    def __rsub__(self, other):
        return -self + other

    def __pow__(self, other):
        other = other if isinstance(other, BPVal) else BPVal(other)
        newValue = BPVal(self.value ** other.value)
        newValue._previous = [self, other]

        def backprop():
            self.grad += other.value * self.value ** (other.value - 1) * newValue.grad  
            #other.grad += log(self.value) * (self.value ** other.value) * newValue.grad
        newValue._backpropagation = backprop

        return newValue

    def __rpow__(self, other):
        return BPVal(other) ** self

    def __neg__(self):
        return -1*self

    def relu(self):
        newValue = BPVal(0) if self.value <= 0 else BPVal(self.value)
        newValue._previous = [self]

        def backprop():
            self.grad += (self.value > 0) * newValue.grad   #relu(x)` = 1 if x > 0 else 0
        newValue._backpropagation = backprop

        return newValue

    def sigmoid(self):
        newValue = BPVal(1/(1+math.exp(-self.value)))
        newValue._previous = [self]

        def backprop():
            self.grad += newValue.value * (1 - newValue.value) * newValue.grad
        newValue._backpropagation = backprop

        return newValue

    def backward(self):
        visited = set()
        topsorted = []
        def topological_sort(start):
            if start not in visited:
                visited.add(start)
                for p in start._previous:
                    topological_sort(p)
                topsorted.append(start)

        topological_sort(self)
        self.grad = 1.0
        for node in reversed(topsorted):
            node._backpropagation()
            

    def params(self):
        params = []
        for p in self._previous:
            params.append(p)
            params.extend(p.params())
        return params

    def __iadd__(self, other):
        if isinstance(other, BPVal):
            self.value += other.value
        else:
            self.value += other
        return self